In [1]:
import requests
import subprocess
from dateutil import parser
from icalendar import Calendar
from datetime import datetime, timezone, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from icalendar import Calendar, Event
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import pytz


In [2]:
url = "https://corsi.unibo.it/2cycle/StatisticalSciences/timetable?anno=1&curricula=A91-000"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

cookies_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, 'ACCEPT ALL COOKIES')))
cookies_button.click()
_30_days_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fc-monthSwitch-button.fc-button.fc-state-default.fc-corner-right')))
_30_days_button.click()


In [3]:
# Read the existing calendar file
with open('Università.ics', 'rb') as f:
    cal = Calendar.from_ical(f.read())

for i in range(4):
    print(i)
    next_30_days_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fc-next-button.fc-button.fc-state-default')))
    next_30_days_button.click()
    sleep(5)
    table = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fc-list-table ')))
    rows = table.find_elements(By.TAG_NAME, 'tr')
    print(len(rows))
    for row in rows:

        # Create a new Event object
        event = Event()

        # getting the class of the object
        html_class = row.get_dom_attribute('class')
        if 'heading' in html_class:
            date = row.text
            print('-'*40)
        elif 'item' in html_class:
            time = row.find_elements(By.TAG_NAME, 'td')[0]
            lecture_data = row.find_elements(By.TAG_NAME, 'td')[-1]

            start_time = date + ' ' + time.text.split(' - ')[0]
            start_time = parser.parse(start_time)
            end_time = date + ' ' + time.text.split(' - ')[1]
            end_time = parser.parse(end_time)

            try:
                lecture_period = lecture_data.find_elements(By.TAG_NAME, 'p')[0].text
            except:
                lecture_period = 'no data on lecture period'

            try:
                lecture_professor = lecture_data.find_element(By.CLASS_NAME, 'docente').text
            except:
                lecture_professor = 'no data on lecture professor'

            try:
                lecture_name = lecture_data.find_element(By.TAG_NAME, 'span').text
            except:
                lecture_name = 'no data on lecture name'
            
            try:
                lecture_place = lecture_data.find_elements(By.TAG_NAME, 'p')[-1].text
            except:
                lecture_place = 'no data on lecture place'

            print(lecture_name, lecture_professor, lecture_place)
            # adding the lesson
            event.add('summary', lecture_name)
            event.add('dtstart', start_time)
            event.add('dtend', end_time)
            event.add('location', lecture_place)
            event.add('description', lecture_professor + '\n' + lecture_period)

            # Add the event to the calendar
            cal.add_component(event)
  

# Write the updated calendar to a file
with open('Università.ics', 'wb') as f:
    f.write(cal.to_ical())


0
69
----------------------------------------
STOCHASTIC PROCESSES Professor: Pietro Rigo Place: AULA FILOPANTI - Piano Terra - Edificio in Bo viale Quirico Filopanti 5
----------------------------------------
STATISTICAL MODELS AND APPLICATIONS Professor: Giuliano Galimberti Place: AULA 22 - Piano Secondo - Edificio in Bo - p.zza A. Scaravilli 1-2
SUPERVISED STATISTICAL LEARNING Professor: Laura Anderlucci Place: AULA V - Piano Terra - Edificio in Bo - viale Berti Pichat 6-6/2
----------------------------------------
STATISTICAL MODELS AND APPLICATIONS Professor: Giuliano Galimberti Place: AULA 32 - Piano Terzo - Edificio in Bo - p.zza A. Scaravilli 1-2
MEASURE THEORY Professor: Andrea Brini Place: AULA FILOPANTI - Piano Terra - Edificio in Bo viale Quirico Filopanti 5
SUPERVISED STATISTICAL LEARNING Professor: Laura Anderlucci Place: AULA 2 - Piano Terra - Edificio in Bo - p.zza A. Scaravilli 1-2
STOCHASTIC PROCESSES Professor: Pietro Rigo Place: AULA FILOPANTI - Piano Terra - Edific

In [4]:

subprocess.run(['open', 'Università.ics'])


CompletedProcess(args=['open', 'Università.ics'], returncode=0)

In [5]:
# # Read the iCalendar file
# with open('UNI.ics', 'rb') as f:
#     cal = Calendar.from_ical(f.read())

# # Specify the date after which events should be displayed
# cutoff_date = datetime(2023, 1, 1, tzinfo=pytz.timezone('Europe/Rome'))

# # Iterate over the events in the calendar
# for event in cal.walk('vevent'):
#     # Get the start date of the event
#     start_date = event.get('dtstart').dt
#     start_datetime = datetime.combine(start_date, datetime.min.time())
#     start_datetime = start_datetime.astimezone(timezone(timedelta(hours=1)))

#     # If the start date is before the cutoff date, print the event

#     print(event.get('summary') + ' here')

#     # Print the start and end dates and times of the event
#     print(event.get('dtstart').dt)
#     print(event.get('dtend').dt)
#     # try:
#     #     if start_datetime >= cutoff_date:
#     #         print('-'*100)
#     #         cal.subcomponents.remove(event)
#     # except:

#     #     print(event.get('summary'))

#     #     # Print the start and end dates and times of the event
#     #     print(event.get('dtstart').dt)
#     #     print(event.get('dtend').dt)

In [6]:
# Read the iCalendar file
with open('Università.ics', 'rb') as f:
    cal = Calendar.from_ical(f.read())

# Specify the date after which events should be removed
cutoff_date = datetime(2023, 1, 1, tzinfo=pytz.timezone('Europe/Rome'))

# Iterate over the events in the calendar
for event in cal.walk('vevent'):
    # Get the start date of the event
    start_date = event.get('dtstart').dt
    start_datetime = datetime.combine(start_date, datetime.min.time())
    start_datetime = start_datetime.astimezone(timezone(timedelta(hours=1)))
    
    # If the start date is after the cutoff date, remove the event from the calendar
    if start_datetime > cutoff_date:
        cal.subcomponents.remove(event)

In [7]:
# Write the modified calendar to a new file
with open('Università.ics', 'w') as f:
    f.write(cal.to_ical().decode('utf-8'))


subprocess.run(['open', 'Università.ics'])

CompletedProcess(args=['open', 'Università.ics'], returncode=0)